In [10]:
import time
import pyscf
import numpy as np
import os
from pyscf import lib
from gpu4pyscf.dft import rks
from monty.json import jsanitize
# from gpu4pyscf import dft
# from pyscf.geomopt.geometric_solver import optimize

from ase.io import read, write
from ase import Atoms
from ase.io import read, write
from ase.io import Trajectory

from sella import Sella
from reactML.common.pyscf2ase import ase_to_string, PySCF_calculator

import argparse
import logging

lib.num_threads(8)


import numpy as np
from ase import Atoms
import os

In [11]:
def perturb_atoms(atoms, amplitude=0.01):
    """
    Add random perturbations to atomic positions in an ASE Atoms object.
    
    Parameters:
    -----------
    atoms : ase.Atoms
        The atomic structure to perturb
    amplitude : float, optional
        Maximum displacement amplitude in Angstroms (default: 0.01)
        
    Returns:
    --------
    ase.Atoms
        New Atoms object with perturbed positions
    """
    
    # Create a copy of the original atoms object
    perturbed_atoms = atoms.copy()
    
    # Get the number of atoms
    n_atoms = len(atoms)
    
    # Generate random displacements for each atom in 3D
    # Using normal distribution with mean 0 and std = amplitude/3
    # (this ensures most perturbations are within ±amplitude)
    displacements = np.random.normal(0, amplitude/3, size=(n_atoms, 3))
    
    # Get original positions
    positions = perturbed_atoms.get_positions()
    
    # Add displacements to positions
    new_positions = positions + displacements
    
    # Set new positions
    perturbed_atoms.set_positions(new_positions)
    
    return perturbed_atoms


def get_mfGPU(mol):

    xc = 'wb97xd'
    # auxbasis = 'def2-svp-jkfit' #  need to find the right auxbasis
    # auxbasis = 'def2-tzvpp-jkfit'
    scf_tol = 1e-10
    max_scf_cycles = 200
    screen_tol = 1e-14
    grids_level = 3
        
    mol.verbose = 1
    mf_GPU = rks.RKS(mol, xc=xc, disp='d3bj').density_fit()
    mf_GPU.grids.level = grids_level
    mf_GPU.conv_tol = scf_tol
    mf_GPU.max_cycle = max_scf_cycles
    mf_GPU.screen_tol = screen_tol

    mf_GPU = mf_GPU.PCM()
    mf_GPU.verbose = 1 # 6 for details
    mf_GPU.grids.atom_grid = (99,590)
    mf_GPU.small_rho_cutoff = 1e-10
    mf_GPU.with_solvent.lebedev_order = 29 # 302 Lebedev grids
    mf_GPU.with_solvent.method = 'COSMO' # 'IEF-PCM' # C-PCM, SS(V)PE, COSMO
    mf_GPU.with_solvent.eps = 78.3553 # for water
    # mf_GPU.with_solvent.method = 'IEF-PCM'
    # mf_GPU.with_solvent.solvent = 'water'
    mf_GPU.kernel()
    
    return mf_GPU


In [12]:
basis = 'def2-svpd'
max_memory = 32000
delta0 = 0.1
threepoint = True

In [13]:
atom_path = os.path.join('./', 's.xyz')
atoms = read(atom_path)

atoms = perturb_atoms(atoms, amplitude = 0.05)

atoms_string = ase_to_string(atoms)
calculator = PySCF_calculator(mf_class = get_mfGPU)

atoms.set_calculator(calculator)

print(atoms.get_potential_energy())
print(atoms.get_forces())

/home/zhongpc/anaconda3/envs/pyscf/lib/python3.9/site-packages/ase/calculators/calculator.py:637: FutureWarning: The keyword "ignore_bad_restart_file" is deprecated and will be removed in a future version of ASE.  Passing more than one positional argument to Calculator is also deprecated and will stop functioning in the future.  Please pass arguments by keyword (key=value) except optionally the "restart" keyword.
  warnings.warn(
/tmp/ipykernel_353767/4171857915.py:9: DeprecationWarning: Please use atoms.calc = calc
  atoms.set_calculator(calculator)


-1011.4757380216074
[[-0.04927105 -0.02372645  0.03594101]
 [ 0.00264112 -0.00329989  0.00373777]
 [ 0.03505878  0.03067451 -0.03147306]
 [ 0.01380582  0.00207521 -0.0077748 ]
 [ 0.00232535  0.00208613  0.00217502]
 [-0.04711892 -0.02674533  0.01811582]
 [ 0.00275115  0.00109114  0.00061787]
 [ 0.00528462 -0.00710613 -0.00948058]
 [ 0.02750054  0.01822229 -0.01848715]
 [ 0.00702259  0.00672851  0.00662807]]


In [14]:
execute_path = os.path.join('./', 'execution_opt.log')
logfile_path = os.path.join('./', 'sella_opt.log')
traj_path = os.path.join('./', 'sella_opt.traj')

if os.path.exists(logfile_path):
    os.remove(logfile_path)
if os.path.exists(traj_path):
    os.remove(traj_path)
if os.path.exists(execute_path):
    os.remove(execute_path)


# Configure logging
logging.basicConfig(
    filename = execute_path,
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Create trajectory file
traj = Trajectory(traj_path , 'w')

# Get initial energy and forces
energy = atoms.get_potential_energy()
forces = atoms.get_forces()
fmax = max(np.abs(forces.flatten()))

print("Initial config:")
print(f"Energy: {energy:.6f}")
print(f"Max force: {fmax:.6f}")


Initial config:
Energy: -1011.475738
Max force: 0.049271


In [15]:
# # compute the Heassian matrix of the optimized geometry directly using PySCF
# atom_string = ase_to_string(atoms)
# mol = pyscf.M(atom=atom_string, basis= basis, max_memory= max_memory)
# mf_GPU = get_mfGPU(mol)

# e_dft = mf_GPU.kernel()

# # Compute the Hessian
# h = mf_GPU.Hessian() 
# h.auxbasis_response = 2

# h_dft = h.kernel()


In [16]:
def hessian_function(atoms):
    atom_string = ase_to_string(atoms)
    mol = pyscf.M(atom=atom_string, basis= basis, max_memory= max_memory)
    mf_GPU = get_mfGPU(mol)

    # Compute the Hessian
    h = mf_GPU.Hessian() 
    h.auxbasis_response = 2

    h_dft = h.kernel()
    natm = h_dft.shape[0]
    h_dft_reshape = h_dft.copy()
    h_dft_reshape = h_dft_reshape.transpose([0,2,1,3]).reshape([3*natm,3*natm])

    return h_dft_reshape
    

In [17]:

# Create Sella optimizer
opt = Sella(
    atoms,
    gamma = 0.1,             # convergence criterion for iterative diagonalization
    delta0 = delta0,         # initial trust radius
    logfile= logfile_path,
    order = 1,
    threepoint = threepoint,
    sigma_inc = 1.15, # default to be 1.15
    sigma_dec = 0.65, # default to be 0.65
    diag_every_n = 40,
    nsteps_per_force_diag = 30, # when force is small, update the Hessian every n steps
    check_nsteps = [5, 10], # update the Hessian at the given steps
    eta = 1e-6,
    hessian_function = hessian_function
)

In [18]:
for step, _ in enumerate(opt.irun(fmax= 0.0005, steps= 200)):
    # Save current configuration to trajectory
    # atoms_tosave = atoms.copy()
    # atoms_tosave.calc = None
    # traj.write(atoms_tosave)

    hessian = opt.pes.H
    eigen_values = hessian.evals
    print(eigen_values)

    # if step == 5:
    #     break

None
[-9.86891708e-02 -1.19146249e-02 -8.42232858e-03 -2.23396494e-03
 -1.02066407e-03  2.78172342e-04  1.36699604e-03  2.29212508e-03
  4.58628587e-03  8.09157199e-03  8.80147148e-03  1.36153481e-02
  1.55069325e-02  3.98081178e-02  6.31490663e-02  7.75098251e-02
  1.12157455e-01  1.62534180e-01  1.88459548e-01  2.15264662e-01
  2.37623595e-01  2.46731281e-01  2.76731977e-01  3.08571634e-01
  4.61021885e-01  7.22958092e-01  7.34203222e-01  1.03109916e+00
  1.35698348e+00  2.82631856e+00]
[-9.80527250e-02 -1.18658034e-02 -8.41739297e-03 -2.23431253e-03
 -1.02302569e-03  2.79382084e-04  1.36663933e-03  2.29402745e-03
  4.57760846e-03  8.21877741e-03  9.07135510e-03  1.38072340e-02
  1.55512940e-02  3.98042926e-02  6.36853225e-02  7.81970065e-02
  1.12147294e-01  1.62647705e-01  1.88622305e-01  2.15086181e-01
  2.37071863e-01  2.48247154e-01  2.77095368e-01  3.58216899e-01
  4.44147755e-01  7.26779086e-01  7.46892734e-01  1.02871268e+00
  1.27975625e+00  2.60497999e+00]
[-1.02462758e-01 